In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets, models
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
 
device = torch.device('cuda:0')
torch.manual_seed(37)
np.random.seed(37)

In [2]:
train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize((227,227)),             
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
 
test_transform = transforms.Compose([
        transforms.Resize((227,227)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

product_dict = ['dress','big dress','blouse','longsleeve blouse','denim jeans','ring','earring',
               'cap','purse','bag','phone cover','phone','clock','some baby shit','rice cooker',
               'coffee beans',"women's shoes",'heels','electronics','thumbdrives','chairs?',
               'racket','helmets','gloves','watches','belts','earphone/headphones','toy cars',
               'jacket',"men's wear??","men's shoes",'confectionary','masks','sanitizers',
               'beauty products','perfume','cleaning equipment?','laptops','kitchen stuff',
               'gardening stuff','bathroom','sofas and pillows']

In [3]:
'''train_csv = pd.read_csv("I:/Downloads/shopee-product-detection-dataset-002/train.csv")
train_csv['fileloc'] = "L:/Downloads/shopee-product-detection-dataset-002/train/train/"\
+ train_csv['category'].apply(folderwrite) + '/' + train_csv['filename']'''

'train_csv = pd.read_csv("I:/Downloads/shopee-product-detection-dataset-002/train.csv")\ntrain_csv[\'fileloc\'] = "L:/Downloads/shopee-product-detection-dataset-002/train/train/"+ train_csv[\'category\'].apply(folderwrite) + \'/\' + train_csv[\'filename\']'

In [4]:
root = "I:/shopee-product-detection-dataset-002/train/train/"
master_data = datasets.ImageFolder(root, transform=train_transform)
n = len(master_data)
n_test = int(n/16)
batch_size = 30
train_data, test_data = random_split(master_data, (n - n_test, n_test))
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=False, num_workers=10)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, pin_memory=False, num_workers=8)

In [5]:
print(len(train_loader), len(test_loader))

3264 218


for images,labels in train_loader: 
    break

print('Label:', labels.numpy())

im = make_grid(images, nrow=5)
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)
im_inv = inv_normalize(im)

plt.figure(figsize=(200,120))
plt.imshow(np.transpose(im_inv.numpy(), (1, 2, 0)));

In [6]:
model = models.resnet101(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

In [7]:
model.fc = nn.Sequential(nn.Linear(2048, 2048),
                         nn.ReLU(inplace=True),
                         nn.Linear(2048, 42),
                         nn.LogSoftmax(dim=1))

In [8]:
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.00002)

In [10]:
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>8}')
    print(f'________\n{sum(params):>8}')

In [11]:
count_parameters(model)

 4194304
    2048
   86016
      42
________
 4282410


In [ ]:
import time
start_time = time.time()

epochs = 6
#max_trn_batch = 50
max_trn_batch = len(train_loader)
max_tst_batch = len(test_loader)

train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    trn_corr = 0
    tst_corr = 0
    total_loss = 0
    total_loss_tr = 0
    mid_time = time.time()
    #fetch_time = time.time()
    
    for b, (X_train, y_train) in enumerate(train_loader):
        if b == max_trn_batch:
            break
        b += 1
        X_train = X_train.to(device, non_blocking=True)
        y_train = y_train.to(device, non_blocking=True)
        #print('fetch time: ', (time.time()-fetch_time))
        #gpu_time = time.time()
        
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        total_loss_tr += loss.item()
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('gpu time: ', (time.time() - gpu_time))
        if b%300==0:
            print(f'epoch:{i+1:2}  batch: {batch_size*b:5}/{batch_size*max_trn_batch}  \
train loss: {total_loss_tr/300:5.3f}  train accuracy: {trn_corr*100/7200:7.3f}%  \
time: {time.time() - mid_time:5.3f} seconds')
            trn_corr = 0
            total_loss_tr = 0
            mid_time = time.time()
        #fetch_time = time.time()
    #train_losses.append(loss)
    #train_correct.append(trn_corr)

    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            if b == max_tst_batch:
                break
            b += 1
            X_test = X_test.cuda()
            y_val = model(X_test).to(torch.device('cpu'))

            predicted = torch.max(y_val.data, 1)[1] 
            tst_corr += (predicted == y_test).sum()
            total_loss += criterion(y_val, y_test)
    
        print(f'epoch {i+1} test loss: {total_loss/b:5} \
test accuracy: {tst_corr.item()*100/(batch_size*b):7.3f}%\n')
    
print(f'\nDuration: {(time.time() - start_time)/60:.2f} minutes')

epoch: 1  batch:  9000/97920  train loss: 3.309  train accuracy:  37.056%  time: 42.780 seconds
epoch: 1  batch: 18000/97920  train loss: 2.483  train accuracy:  63.000%  time: 26.666 seconds
epoch: 1  batch: 27000/97920  train loss: 1.997  train accuracy:  70.319%  time: 26.751 seconds
epoch: 1  batch: 36000/97920  train loss: 1.764  train accuracy:  72.889%  time: 27.014 seconds
epoch: 1  batch: 45000/97920  train loss: 1.647  train accuracy:  74.333%  time: 27.212 seconds
epoch: 1  batch: 54000/97920  train loss: 1.544  train accuracy:  76.347%  time: 27.135 seconds
epoch: 1  batch: 63000/97920  train loss: 1.491  train accuracy:  76.569%  time: 27.007 seconds
epoch: 1  batch: 72000/97920  train loss: 1.456  train accuracy:  76.736%  time: 28.156 seconds
epoch: 1  batch: 81000/97920  train loss: 1.405  train accuracy:  78.208%  time: 30.526 seconds
epoch: 1  batch: 90000/97920  train loss: 1.395  train accuracy:  78.472%  time: 30.399 seconds
epoch 1 test loss: 1.3356863260269165 te

Batch size 44
Mid layer 3096
Learning Rate 0.0001
Rand seeds 11
epoch 4 test loss: 1.500015139579773 test accuracy:  61.439%

Batch size 44
Mid layer 3096
Learning Rate 0.0001
Rand seeds 12
epoch 3 test loss: 1.4558815956115723 test accuracy:  62.182%

Batch size 36
Mid layer 3096
Learning Rate 0.00005
Rand seeds 12
epoch 4 test loss: 1.3936330080032349 test accuracy:  62.933%

Batch size 36
Mid layer 2096
Learning Rate 0.00005
Rand seeds 12
epoch 4 test loss: 1.4185611009597778 test accuracy:  62.371%

Batch size 24
Mid layer 4096
Learning Rate 0.00002
Rand seeds 15
epoch 6 test loss: 1.3716551065444946 test accuracy:  64.288%

Batch size 24
Learning Rate 0.00002
epoch 8 test loss: 1.2389650344848633 test accuracy:  65.985%

RS 35
Batch size 32
LR 0.00002
epoch 7 test loss: 1.1721174716949463 test accuracy:  67.448%

0 workers: 80 seconds 54% cpu
4 workers: 23.0 seconds 35% cpu
8 workers: 14.5 seconds 68% cpu
11 wprlers: 12.6 seconds 84% cpu
14 workers: 11.6 seconds 98% cpu


In [ ]:
#torch.save(model.state_dict(), "L:/ResNet101_Model.pth")